In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [12]:
filename = "/Users/danasio/Desktop/ADHOC/transaction-data-adhoc-analysis.json"

raw_df = pd.read_json(filename)
raw_df

,address,birthdate,mail,name,sex,username,transaction_items,transaction_value,transaction_date
0,USNS Wagner\nFPO AE 00917,1926/03/11,watsonbrenda@gmail.com,Jennifer Campbell,F,veronicawells,"Exotic Extras,Beef Chicharon,(x4)",5196,2022/01/12
1,USCGC Benson\nFPO AP 48794,2016/07/08,moorelisa@gmail.com,Melissa Kim,F,cgilbert,"Exotic Extras,Beef Chicharon,(x3);HealthyKid 3...",12056,2022/01/12
2,USCGC Benson\nFPO AP 48794,2016/07/08,moorelisa@gmail.com,Melissa Kim,F,cgilbert,"HealthyKid 3+,Gummy Vitamins,(x3);HealthyKid 3...",5500,2022/01/08
3,USCGC Benson\nFPO AP 48794,2016/07/08,moorelisa@gmail.com,Melissa Kim,F,cgilbert,"HealthyKid 3+,Yummy Vegetables,(x1)",500,2022/01/10
4,"33911 Hansen Loaf\nWest Sandra, ND 54188",1949/09/27,michael78@yahoo.com,Kristen Cooper,F,qhernandez,"Candy City,Orange Beans,(x3);Candy City,Gummy ...",2697,2022/01/24
...,...,...,...,...,...,...,...,...,...
83031,"96081 Harrison Brooks\nJenniferbury, MA 02124",2019/10/26,christina55@hotmail.com,Tammy Byrd,F,kmcconnell,"Exotic Extras,Kimchi and Seaweed,(x2);HealthyK...",3588,2022/06/05
83032,428 Kristopher Curve Apt. 206\nMcculloughville...,1913/11/04,melvinschmidt@hotmail.com,Donald Andersen,M,llowe,"HealthyKid 3+,Yummy Vegetables,(x1);Exotic Ext...",2095,2022/06/12
83033,428 Kristopher Curve Apt. 206\nMcculloughville...,1913/11/04,melvinschmidt@hotmail.com,Donald Andersen,M,llowe,"Candy City,Orange Beans,(x4);Candy City,Gummy ...",3396,2022/06/15
83034,428 Kristopher Curve Apt. 206\nMcculloughville...,1913/11/04,melvinschmidt@hotmail.com,Donald Andersen,M,llowe,"HealthyKid 3+,Nutrional Milk,(x2);Candy City,G...",4280,2022/06/09


In [13]:
import string
values_df = raw_df.copy()

values_df = values_df.assign(transaction_items=values_df.transaction_items.str.split(";"))
values_df = values_df[values_df["transaction_items"].apply(len)==1]

values_df["transaction_items"] = values_df["transaction_items"].apply(lambda lst: lst[0])
values_df['amount'] = values_df["transaction_items"].apply(lambda x: int(x.split(',')[-1].strip('(x)')))
values_df["transaction_items"] = values_df["transaction_items"].apply(lambda lst: lst[:-5])

values_df = values_df.drop_duplicates(subset="transaction_items", keep="first")

labels = list(values_df['transaction_items'].values)
values_df = values_df['transaction_value'] / values_df['amount']

values_df.index = labels
values_df

Exotic Extras,Beef Chicharon        1299.0
HealthyKid 3+,Yummy Vegetables       500.0
Exotic Extras,Kimchi and Seaweed     799.0
Candy City,Gummy Worms               150.0
HealthyKid 3+,Gummy Vitamins        1500.0
HealthyKid 3+,Nutrional Milk        1990.0
Candy City,Orange Beans              199.0
dtype: float64

In [14]:
parsed_df = raw_df.assign(transaction_items=raw_df.transaction_items.str.split(";")).explode('transaction_items').reset_index(drop=True)

parsed_df['amount'] = parsed_df["transaction_items"].apply(lambda x: int(x.split(',')[-1].strip('(x)')))
parsed_df["transaction_items"] = parsed_df["transaction_items"].apply(lambda lst: lst[:-5])

parsed_df['transaction_value'] = parsed_df['amount'] * parsed_df['transaction_items'].apply(lambda x: values_df.loc[x])

parsed_df['transaction_date'] = pd.to_datetime(parsed_df["transaction_date"], format="%Y/%m/%d")
parsed_df['Month'] = parsed_df['transaction_date'].dt.month

parsed_df

,address,birthdate,mail,name,sex,username,transaction_items,transaction_value,transaction_date,amount,Month
0,USNS Wagner\nFPO AE 00917,1926/03/11,watsonbrenda@gmail.com,Jennifer Campbell,F,veronicawells,"Exotic Extras,Beef Chicharon",5196.0,2022-01-12,4,1
1,USCGC Benson\nFPO AP 48794,2016/07/08,moorelisa@gmail.com,Melissa Kim,F,cgilbert,"Exotic Extras,Beef Chicharon",3897.0,2022-01-12,3,1
2,USCGC Benson\nFPO AP 48794,2016/07/08,moorelisa@gmail.com,Melissa Kim,F,cgilbert,"HealthyKid 3+,Nutrional Milk",7960.0,2022-01-12,4,1
3,USCGC Benson\nFPO AP 48794,2016/07/08,moorelisa@gmail.com,Melissa Kim,F,cgilbert,"Candy City,Orange Beans",199.0,2022-01-12,1,1
4,USCGC Benson\nFPO AP 48794,2016/07/08,moorelisa@gmail.com,Melissa Kim,F,cgilbert,"HealthyKid 3+,Gummy Vitamins",4500.0,2022-01-08,3,1
...,...,...,...,...,...,...,...,...,...,...,...
166114,428 Kristopher Curve Apt. 206\nMcculloughville...,1913/11/04,melvinschmidt@hotmail.com,Donald Andersen,M,llowe,"HealthyKid 3+,Nutrional Milk",3980.0,2022-06-09,2,6
166115,428 Kristopher Curve Apt. 206\nMcculloughville...,1913/11/04,melvinschmidt@hotmail.com,Donald Andersen,M,llowe,"Candy City,Gummy Worms",300.0,2022-06-09,2,6
166116,"PSC 1089, Box 3720\nAPO AE 08973",2012/04/30,xjones@hotmail.com,Stephanie Russell,F,cmcdowell,"HealthyKid 3+,Nutrional Milk",1990.0,2022-06-03,1,6
166117,"PSC 1089, Box 3720\nAPO AE 08973",2012/04/30,xjones@hotmail.com,Stephanie Russell,F,cmcdowell,"Candy City,Gummy Worms",450.0,2022-06-03,3,6


# Monthly Total Item Count and Sales

In [5]:
monthly_df = parsed_df[['transaction_value','amount']].groupby([parsed_df.transaction_items, parsed_df.transaction_date.dt.month]).agg(sum)

monthly_df.rename(columns={"transaction_value":"Sales","amount":"Count"},inplace=True)
monthly_df.index.names = ['Item', 'Month']

monthly_df

Sales  Count
Item                             Month                   
Candy City,Gummy Worms           1       1433850.0   9559
                                 2       1499400.0   9996
                                 3       1497900.0   9986
                                 4       1506450.0  10043
                                 5       1470150.0   9801
                                 6       1490100.0   9934
Candy City,Orange Beans          1       1945026.0   9774
                                 2       1997363.0  10037
                                 3       1912589.0   9611
                                 4       1972886.0   9914
                                 5       1982836.0   9964
                                 6       2011094.0  10106
Exotic Extras,Beef Chicharon     1      12554835.0   9665
                                 2      12991299.0  10001
                                 3      12750984.0   9816
                                 4      12847110.0   9890
                                 5      13026372.0  10028
                                 6      12862698.0   9902
Exotic Extras,Kimchi and Seaweed 1       7731124.0   9676
                                 2       7949251.0   9949
                                 3       7963633.0   9967
                                 4       7926879.0   9921
                                 5       7808627.0   9773
                                 6       8073096.0  10104
HealthyKid 3+,Gummy Vitamins     1      14521500.0   9681
                                 2      14970000.0   9980
                                 3      15217500.0  10145
                                 4      14763000.0   9842
                                 5      14922000.0   9948
                                 6      14970000.0   9980
HealthyKid 3+,Nutrional Milk     1      19356730.0   9727
                                 2      19285090.0   9691
                                 3      19653240.0   9876
                                 4      19474140.0   9786
                                 5      19663190.0   9881
                                 6      19436330.0   9767
HealthyKid 3+,Yummy Vegetables   1       4979500.0   9959
                                 2       5128000.0  10256
                                 3       4948000.0   9896
                                 4       4930500.0   9861
                                 5       4867500.0   9735
                                 6       4861000.0   9722

In [6]:
customers_df = parsed_df[['name','amount','Month']]
customers_df = pd.pivot_table(
    customers_df,
    values='amount', 
    index=['Month'],
    columns=['name'],
    aggfunc=sum,
    fill_value = 0
)

customers_df

name,Aaron Beasley,Aaron Brewer,Aaron Brown,Aaron Coffey,Aaron Davis,Aaron Escobar,Aaron Flores,Aaron Flowers,Aaron Garrett,Aaron Holder,...,Zachary Richard,Zachary Salinas,Zachary Scott,Zachary Smith,Zachary Torres,Zachary Valentine,Zachary Ware,Zachary Williams,Zachary Wilson,Zachary York
Month,,,,,,,,,,,,,,,,,,,,,
1,0,4,0,1,6,0,11,11,8,0,...,17,0,13,16,0,7,6,24,0,8
2,0,6,0,7,1,11,15,3,15,13,...,15,1,15,10,0,6,9,23,0,17
3,0,1,20,9,0,0,5,0,6,9,...,14,0,3,19,14,3,16,19,0,7
4,9,9,14,12,15,1,8,5,0,4,...,27,5,15,31,6,2,5,14,4,12
5,8,0,8,12,15,7,0,0,7,8,...,8,13,9,31,7,14,10,0,4,23
6,1,13,1,0,12,0,1,16,7,6,...,10,4,5,40,9,19,8,0,10,5


In [10]:
inactives = customers_df.apply(lambda x: [0 if i==1 else ((1 if x[i]==0 else 0) if any(x[:i]) else 0) for i in x.index]).transpose().sum()
engaged = customers_df.apply(lambda x: [1 if all(x[:i]) else 0 for i in x.index]).transpose().sum()
repeaters = customers_df.apply(lambda x: [0 if i==1 else (1 if x[i-1] and x[i] else 0)for i in x.index]).transpose().sum()

customer_categories_df = pd.DataFrame({
    'repeater': repeaters,
    'inactive': inactives,
    'engaged' : engaged
})

customer_categories_df

,repeater,inactive,engaged
Month,,,
1,0,0,6588
2,5172,1416,5172
3,5216,1747,4126
4,5154,1909,3289
5,5110,1917,2667
6,5193,1835,2190
